## Day 23

https://adventofcode.com/2020/day/23

In [1]:
TEST_DATA = '389125467'

In [2]:
DATA = '589174263'

In [3]:
class Node:
    __slots__ = ('value', 'prev', 'succ')
    
    def __init__(self, *, value, prev, succ):
        self.value = value
        self.prev = prev
        self.succ = succ
        
    def __repr__(self):
        return f'<{self.__class__.__name__}({self.value})>'
    
    def __str__(self):
        return str(self.value)

In [4]:
class IndexedCircularLinkedList:
    def __init__(self, data=None):
        self.head = None
        self.index = {}
        if data is not None:
            for value in data:
                self.append(value)
    
    def __len__(self):
        return len(self.index)
    
    def __iter__(self):
        if self.head is None:
            return iter(())
        start = self.head
        node = start
        while node is not None:
            yield node
            node = node.succ if node.succ is not start else None
    
    def __reversed__(self):
        if self.head is None:
            return iter(())
        start = self.head.prev
        node = start
        while node is not None:
            yield node
            node = node.prev if node.prev is not start else None
            
    def __repr__(self):
        if len(self) == 9:
            return f'<{self.__class__.__name__}({list(self)})'
        return f'<{self.__class__.__name__} with {len(self)} items>'
    
    def __str__(self):
        if len(self) == 9:
            return str(list(node.value for node in self))
        return repr(self)
    
    def find(self, item):
        return self.index[item]
    
    def append(self, value):
        node = Node(value=value, prev=None, succ=None)
        if not self.head:
            node.prev = node.succ = node
            self.head = node
        else:
            tail = self.head.prev
            tail.succ = node
            self.head.prev = node
            node.prev = tail
            node.succ = self.head
        self.index[value] = node
        
    def remove(self, value):
        node = self.index[value]
        if node is self.head:
            self.head = node.succ
        node.prev.succ = node.succ
        node.succ.prev = node.prev
        del self.index[value]
        if not self.index:
            self.head = None
            
    def insert(self, after_value, value):
        after_node = self.index[after_value]
        node = Node(value=value, prev=after_node, succ=after_node.succ)
        after_node.succ = node
        node.succ.prev = node
        self.index[value] = node

### Solution to Part 1

In [5]:
class CupPart1Adapter:
    def __init__(self, numbers, *, max_label=None):
        assert max_label is None
        self.number = len(numbers)
        self.labels = list(numbers)
        self.current = 0
        self.current_label = self.labels[self.current]
        self.max_label = max(self.labels)
    
    def __repr__(self):
        return (
            f'{self.__class__.__name__}('
            f'{self.labels!r}'
            f', max_label={self.max_label})'
        )

    def pick(self, *, n):
        return [
            self.labels[(self.current + k) % self.number]
            for k in range(1, n+1)
        ]
    
    def remove(self, *, picked):
        for label in picked:
            index = self.labels.index(label)
            self.labels.pop(index)

    def insert(self, *, destination_label, picked):
        destination = self.labels.index(destination_label)
        insert_at = (destination + 1) % self.number
        for label in reversed(picked):
            self.labels.insert(insert_at, label)

    def update_current(self):
        k = self.labels.index(self.current_label)
        self.current = (k + 1) % self.number
        self.current_label = self.labels[self.current]

In [6]:
class CupSimulator:
    def __init__(
            self,
            numbers,
            *,
            adapter_type,
            max_label=None, 
    ):
        assert len(numbers) == 9
        self.adapter_type = adapter_type
        self.max_label = max_label
        self.adapter = adapter_type(numbers, max_label=max_label)
    
    @classmethod
    def from_str(cls, data, *, adapter_type, max_label=None):
        return cls(
            [int(c) for c in data],
            adapter_type=adapter_type,
            max_label=max_label,
        )

    @property
    def current_label(self):
        return self.adapter.current_label
    
    @property
    def labels(self):
        return self.adapter.labels
    
    def __repr__(self):
        return (
            f'{self.__class__.__name__}('
            f'{self.labels!r}'
            f', adapter_type={self.adapter_type}'
            f', max_label={self.max_label}'
            f')'
        )

    def find_destination_label(self, *, picked):
        cups = self.adapter
        label = cups.current_label - 1
        while label in picked or label < 1:
            label -= 1
            if label < 1:
                label = cups.max_label
        return label
    
    def move(self, *, n=3, verbose=False):
        cups = self.adapter
        picked = cups.pick(n=n)
        destination_label = self.find_destination_label(picked=picked)
        if verbose:
            print(f'pick up: {picked}')
            print(f'destination label: {destination_label}')
            print()
        cups.remove(picked=picked)
        cups.insert(destination_label=destination_label, picked=picked)
        cups.update_current()

    def simulate(self, *, moves, cups=3, verbose=False):
        for move in range(1, moves + 1):
            if verbose:
                print(f'move {move}')
                print(f'current label: {self.current_label}')
                print(f'cups: {self.labels}')
            self.move(n=cups, verbose=verbose)

In [7]:
cups1 = CupSimulator.from_str(DATA, adapter_type=CupPart1Adapter)

In [8]:
cups1.simulate(moves=100)

In [9]:
def part1_answer(cups, *, after=1):
    s = ''.join(map(str, cups.labels))
    i = s.find(str(after))
    return s[(i+1):] + s[:i]

In [10]:
part1_answer(cups1)

'43896725'

### Solution to Part 2

In [11]:
class CupPart2Adapter:
    def __init__(self, numbers, *, max_label=None):
        self.labels = IndexedCircularLinkedList(numbers)
        if max_label is not None:
            self.max_label = max_label
            for label in range(max(numbers) + 1, max_label + 1):
                self.labels.append(label)
        else:
            self.max_label = max(numbers)
        self.number = len(self.labels)
        self.current = self.labels.head
        self.current_label = self.current.value
    
    def __repr__(self):
        return f'<{self.__class__.__name__} with {self.number} cups>'

    def __str__(self):
        assert len(self.labels) == 9
        return str(self.labels)

    def pick(self, *, n):
        node = self.current.succ
        picked = []
        for _ in range(n):
            picked.append(node.value)
            node = node.succ
        return picked
    
    def remove(self, *, picked):
        for label in picked:
            self.labels.remove(label)

    def insert(self, *, destination_label, picked):
        after = destination_label
        for label in picked:
            self.labels.insert(after, label)
            after = label

    def update_current(self):
        self.current = self.current.succ
        self.current_label = self.current.value

In [12]:
cups2 = CupSimulator.from_str(
    DATA,
    max_label=1_000_000,
    adapter_type=CupPart2Adapter)

In [13]:
cups2.simulate(moves=10_000_000)

In [14]:
def part2_answer(cups2, *, after=1):
    node = cups2.labels.find(after)
    return node.succ.value * node.succ.succ.value

In [15]:
part2_answer(cups2)

2911418906